# Project 4 - Algorithm comparison and local minima

In [ ]:
# Lets begin by setting up our toy problem again.
from pyplasmaopt import *
nfp = 2
(coils, currents, magnetic_axis, eta_bar) = get_24_coil_data(nfp=nfp, ppp=10, at_optimum=False)
stellarator = CoilCollection(coils, currents, nfp, True)
iota_target = 0.103
coil_length_target = 4.398229715025710
magnetic_axis_length_target = 6.356206812106860
eta_bar = -2.25
obj = SimpleNearAxisQuasiSymmetryObjective(
        stellarator, magnetic_axis, iota_target, eta_bar=eta_bar,
        coil_length_target=coil_length_target, magnetic_axis_length_target=magnetic_axis_length_target)

## Tasks

1) So far we have always used the BFGS optimisation algorithm. Using the scipy `minimize` function, compare the BFGS algorithm with `CG`, `Nelder-Mead` and `L-BFGS-B` with memory 10 and 1 (scipy doesn't implement pure gradient descent, so we use L-BFGS with a short memory as an approximation). Plot the objective values during the optimisation and compare. To save some time, you can just run each optimisation algorithm for 500 iterations.

2) We can not prove uniqueness of minimizers of our objective, nor do we expect it. To find distinct minimizers, a common strategy is to start the optimisation at different initial guesses. Consider several random perturbations of the initial configuration and check if the optimisation algorithm converges to the same configuration.
_Suggestion_: Consider perturbations of the Fourier coefficients that describe the coils. You can obtain the indices in the dof vector `x` that correspond to the coil coeffients via `idx = list(range(*obj.coil_dof_idxs))`. Draw random numbers in the interval `[-0.1, +0.1]` and perturb the Fourier coefficients with these values.

## Solutions

### Task 1

In [ ]:
from scipy.optimize import minimize

def scipy_fun(x):
    obj.update(x)
    res = obj.res
    dres = obj.dres
    return res, dres

methods = [
    'l-bfgs-b',
    'l-bfgs-b',
    'bfgs',
    'cg',
    'nelder-mead'
]
options = [
    {'maxcor': 10},
    {'maxcor': 1},
    {},
    {},
    {}
]
labels = [
    'L-BFGS-B (10)',
    'L-BFGS-B (1)',
    'BFGS',
    'CG',
    'Nelder-Mead'
]

results = {}
for i in range(len(methods)):
    obj.clear_history()
    res = minimize(scipy_fun, obj.x0, jac=True, method=methods[i], tol=1e-20,
                   options={**{"maxiter": 500}, **options[i]},
                   callback=obj.callback)
    print(res)
    results[labels[i]] = obj.Jvals

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
for l in labels:
    plt.semilogy(results[l], label=l)
plt.legend()
plt.show()

### Task 2

In [ ]:
x0 = obj.x0
x0_perturb = x0.copy()
idx = list(range(*obj.coil_dof_idxs))
np.random.seed(1)
x0_perturb[idx] += np.random.uniform(low=-0.1, high=0.1, size=len(idx))
obj.update(x0_perturb)
plot_stellarator(obj.stellarator)

In [ ]:
obj.clear_history()
res = minimize(scipy_fun, x0_perturb, jac=True, method='bfgs', tol=1e-20,
               options={"maxiter": 500},
               callback=obj.callback)


In [ ]:
plot_stellarator(stellarator) 

Let's run the optimisation from the unperturbed initial guess to see if it looks identical.

In [ ]:
obj.clear_history()
res = minimize(scipy_fun, obj.x0, jac=True, method='bfgs', tol=1e-20,
               options={"maxiter": 500},
               callback=obj.callback)

In [ ]:
plot_stellarator(stellarator) 

*Looks quite different!*